### Stocks Analysis

In [1]:
#Packages
import pandas as pd
import numpy as np
from selenium import webdriver
from datetime import date
import time
import io

In [2]:
#This function generates hyperlinks to stooq.pl website. Ticker, first and last observation date and number of pages need to be provided
#If number of pages won't be provided 20 hyperlinks will be created by default
#(Take into consideration that if more historical data need to be downloaded default no of pages might not be sufficient)
def generateHyperlinks(ticker,startDate = "2020-01-01",endDate = str(date.today()),no_of_pages = 20):
    startDate = startDate.replace("-","")
    endDate = endDate.replace("-","")
    webpage = r"https://stooq.pl/q/d/?s=" + ticker + '&c=0&d1=' + startDate + "&d2=" + endDate + "&l="
    hyperlinks = [webpage + str(i) for i in range(1,no_of_pages+1)]
    return hyperlinks

In [3]:
#Funcion which iterates through websites and save share prices from each page to seperate list argument
#Input: list of hyperlinks to iterate
#Output: list with unordered data collected from each 'subpage'
def pageIterator(hyperlinks):
    driver = webdriver.Chrome(executable_path=r"C:\Users\woote\Desktop\Python\Projects\chromedriver.exe")
    tableValues = []
    #Remember that Stooq.pl has a limit of daily page refreshes
    for i,link in enumerate(hyperlinks):

        driver.get(link)

        if i == 0:
            try:
                accept = driver.find_element_by_xpath("//p[contains(@class, 'fc-button-label')]")
                accept.click()
            except:
                pass
            time.sleep(3)

        try:
            tableText = driver.find_element_by_id("fth1")
            tableValues.append(tableText.text)
        except:
            break

    return tableValues

In [4]:
#Function which converts a list of unordered list of tables extracted from HTML code
#Input: a list of values
#Output: Clean pandas data frame
def dataCleaner(tableValues, colnames = ['id','day','month','year','open','max','min','close','perc_change','abs_change','volume']):
    
    #Dictionary which will help to convert month names to a digits in date column
    months = {"sty": "01","lut": "02","mar": "03","kwi": "04","maj": "05","cze": "06",\
              "lip": "07","sie": "08","wrz": "09","paź": "10","lis": "11","gru": "12"}
    #Joining list elements with new line operator, then deliminating by spaces
    #Dropping the duplicated rows (as we are scraping column headers as well)
    #And erasing first row wiith wrongly assigned headers
    #Deleting rows where 'close' is nan (usually its additional row pointing the date of dividend)
    data = pd.read_csv(io.StringIO('\n'.join(tableValues)), delim_whitespace=True,names=colnames).drop_duplicates().iloc[1:,:]
    data = data[data['close'].notna()]
    data = data.iloc[:-1 , :]
    #Inserting date column
    data.insert(4,'date',pd.to_datetime(data["day"] + "/" + data["month"].map(months) + "/" + data["year"], format='%d/%m/%Y'))
    #Dropping needless columns
    data.drop(['index','id','day','month','year'],axis = 1,inplace = True)
    #Adjusting data types
    data[['max','min','open','close']] = data[['max','min','open','close']].astype(float)
    data['volume'] = data['volume'].str.replace(",","").fillna(0).astype(int)
    data['perc_change'] = data['perc_change'].str.replace("%","").str.replace("+","").astype(float)
    data.sort_values('date',inplace = True)
    data.reset_index(inplace = True)
    
    return data

In [5]:
def saveValues(data_frame,ticker):
    name = 'Share_prices_' + ticker + '.xlsx'
    data_frame.to_excel(name,sheet_name = ticker)

In [8]:
ticker = 'GTS'
hyperlinks = generateHyperlinks(ticker,"2021-01-01",no_of_pages=8)
tableValues = pageIterator(hyperlinks)
df = dataCleaner(tableValues)
saveValues(df,ticker)

KeyError: "['index'] not found in axis"